In [1]:
import mediapipe as mp
import numpy as np
import cv2 as cv
import os

In [2]:
cap=cv.VideoCapture(0)
while True:
    _,frame=cap.read()

    cv.imshow('frame',frame)
    if cv.waitKey(10)==27:
        cap.release()
        break

cv.destroyAllWindows()

In [3]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic
holistic=mp_holistic.Holistic()

In [16]:
cap=cv.VideoCapture(0)
while True:
    _,frame=cap.read()
    # mediapipe operations
    img=cv.cvtColor(frame,cv.COLOR_BGR2RGB)
    results=holistic.process(img)
    img=cv.cvtColor(img,cv.COLOR_RGB2BGR)
    draw_landmarks(img,results)

    cv.imshow('frame',img)
    if cv.waitKey(1)==27:
        cap.release()
        break

cv.destroyAllWindows()

In [4]:

def draw_landmarks(img,results):
    mp_drawing.draw_landmarks(img,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,style1,style2)
    mp_drawing.draw_landmarks(img,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,style1,style2)
    mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,style1,style2)
    mp_drawing.draw_landmarks(img,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS,style1,style2)

In [5]:
style1=mp_drawing.DrawingSpec((71, 237, 212),1,1)
style2=mp_drawing.DrawingSpec((67, 73, 247),2,1)
def extract_landmarks(results):
    face=np.array([[landmark.x,landmark.y,landmark.z] for landmark in results.face_landmarks.landmark]).flatten()  if results.face_landmarks else np.zeros(1404)
    pose=np.array([[landmark.x,landmark.y,landmark.z , landmark.visibility] for landmark in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    rh=np.array([[landmark.x,landmark.y,landmark.z] for landmark in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    lh=np.array([[landmark.x,landmark.y,landmark.z] for landmark in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)

    return np.concatenate([pose,face,lh,rh])

In [7]:
data_dir='data'


In [8]:
no_of_videos=75
no_of_frames=30

In [13]:
action='welcome'
for video in range(no_of_videos):
    os.makedirs(os.path.join(data_dir,action,f'{video}'))

In [17]:
try:
    cap=cv.VideoCapture(0)

    for video in range(33,75):
        for frame in range(no_of_frames):
            _,framee=cap.read()
            # mediapipe operations
            img=cv.cvtColor(framee,cv.COLOR_BGR2RGB)
            results=holistic.process(img)
            
            img=cv.cvtColor(img,cv.COLOR_RGB2BGR)
            
            draw_landmarks(img,results)
            landmarks=extract_landmarks(results)
            if frame==0:  # for the first frame
                cv.putText(img,f'starting collection',(100,200),cv.FONT_HERSHEY_COMPLEX_SMALL,4,(0,0,255),1,cv.LINE_AA)
                cv.putText(img,f'For {action}  Video {video} Frame {frame}',(120,32),cv.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0),1,cv.LINE_AA)
                cv.imshow('image',img)
                cv.waitKey(4000)
            else:
                cv.putText(img,f'For {action}  Video {video} Frame {frame}',(120,32),cv.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0),1,cv.LINE_AA)
                cv.imshow('image',img)

            path=os.path.join(data_dir,action,str(video),f'{frame}.npy')
            np.save(path,landmarks)
                

            
        
            
            if cv.waitKey(1)==27:
                cap.release()
                break
        
except:
        print('error')


cv.destroyAllWindows()

In [18]:
cap.release()